In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def jsonl2df(path):
    with open(path) as fh:
        lines = fh.read().splitlines()

    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']


    df_inter['json_element'].apply(json.loads)

    return pd.json_normalize(df_inter['json_element'].apply(json.loads))

artists = jsonl2df("artists.jsonl")
sessions = jsonl2df("sessions.jsonl")
tracks = jsonl2df("tracks.jsonl")
users = jsonl2df("users.jsonl")

In [16]:
from pprint import pprint
users.columns
genres_count = {}
for user in users.values:
    genres = user[4]
    if genres:
        for genre in genres:
            if genre not in genres_count:
                genres_count[genre] = 0
            genres_count[genre] += 1
genres_count = sorted(genres_count.items(), key=lambda x:x[1], reverse=True)

pprint(genres_count)

[('neo mellow', 12),
 ('country', 10),
 ('rap', 9),
 ('psychedelic rock', 9),
 ('latin pop', 9),
 ('pop rock', 9),
 ('k-pop', 9),
 ('latin', 8),
 ('reggaeton', 8),
 ('ranchera', 8),
 ('canadian pop', 8),
 ('hip hop', 8),
 ('tropical', 8),
 ('norteno', 8),
 ('latin arena pop', 7),
 ('pop', 7),
 ('folk rock', 7),
 ('k-pop boy group', 7),
 ('corrido', 7),
 ('pop urbaine', 7),
 ('dance pop', 6),
 ('hard rock', 6),
 ('rock', 6),
 ('mexican pop', 6),
 ('adult standards', 6),
 ('album rock', 6),
 ('trap latino', 6),
 ('post-teen pop', 6),
 ('r&b', 6),
 ('modern rock', 5),
 ('art rock', 5),
 ('permanent wave', 5),
 ('rock en espanol', 5),
 ('contemporary country', 4),
 ('regional mexican', 4),
 ('electropop', 4),
 ('pop rap', 4),
 ('french hip hop', 4),
 ('pop punk', 4),
 ('metal', 4),
 ('alternative metal', 4),
 ('alternative rock', 3),
 ('classic rock', 3),
 ('urban contemporary', 3),
 ('soft rock', 3),
 ('country road', 2),
 ('uk pop', 2),
 ('post-grunge', 2),
 ('mellow gold', 1),
 ('nu met

In [25]:
# sessions.columns
sessions.head()

track_count = {}
for session in sessions.values:
    track = session[3]
    event_type = session[4]
    if track:
        if track not in track_count:
            track_count[track] = 0
        track_count[track] += 1
track_count = sorted(track_count.items(), key=lambda x:x[1], reverse=True)

pprint(track_count)


[('13sxKGMd5RDQYavHyZ8Flw', 26),
 ('2kQtm92uUcTTIPN5smIxUl', 23),
 ('1d5jcRB7Ri9ZA0sFh1Moke', 20),
 ('2J3XzDLjbXkny8qhr33zTh', 20),
 ('6UO72VSXEONxdfLyABihs9', 20),
 ('6z1TvLTR0oOsWSJfDrERmj', 20),
 ('5s2zbxZI4Ym5LTKwjZHS2V', 19),
 ('59JBsvbxoibwgWLVE61acB', 19),
 ('5oMecwx9BcU6wacLtnBjpo', 19),
 ('6wLwtK3mkAq0zWPpV6QVbF', 19),
 ('2sFp6E7lNBbblJGmUtCfFH', 19),
 ('4A4A6dwl5DqdqrZaEtKGPR', 19),
 ('4j5WelE5mkf9A2jmts729h', 19),
 ('14BgiTqui9Ae1uWVpGGFDI', 19),
 ('5OwponLnwCAYnWql6cwBru', 19),
 ('7LMtp4HBhWLQ2P2POPn6li', 18),
 ('3HUb7feX3gdnzZVEHy5yFj', 18),
 ('0YguyyCOTyujhSDXWyB4W3', 18),
 ('1ClFrQKter80gTUlnr3xzs', 18),
 ('1MiCIztsIHMrVORLJmbiuH', 18),
 ('2L9dBaAmQ5qydXHSz8bN96', 18),
 ('7ii1YH0rOmpoRfH9HyY8To', 18),
 ('4yDTg9PkshhDpmUIVFoYI5', 18),
 ('4fTlLuSt6R3OnjAj0Ty5OC', 18),
 ('5Psnhdkyanjpgc2P8A5TSM', 18),
 ('4CLRLdxUDPNeO1xCW0U2zC', 18),
 ('5BIMPccDwShpXq784RJlJp', 18),
 ('2Rn5VhDVELkuB4BjZEVJKr', 18),
 ('5rnMgctZxjKDNdqrVqg7Bm', 18),
 ('1kBx9VGumfuvlfqdlAGorE', 18),
 ('2wcDfk3

In [32]:
# tracks.columns
tracks.head()
tracks.sort_values(by=['popularity'], ascending=False)

tracks.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22412 entries, 0 to 22411
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                21274 non-null  object 
 1   name              21294 non-null  object 
 2   popularity        21358 non-null  float64
 3   duration_ms       22412 non-null  int64  
 4   explicit          22412 non-null  int64  
 5   id_artist         21278 non-null  object 
 6   release_date      22412 non-null  object 
 7   danceability      22412 non-null  float64
 8   energy            22412 non-null  float64
 9   key               22412 non-null  int64  
 10  loudness          22412 non-null  float64
 11  speechiness       22412 non-null  float64
 12  acousticness      22412 non-null  float64
 13  instrumentalness  22412 non-null  float64
 14  liveness          22412 non-null  float64
 15  valence           22412 non-null  float64
 16  tempo             22412 non-null  float6

In [6]:
artists.columns
# artists.head()
artists.info()

Index(['id', 'name', 'genres'], dtype='object')